### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

#### generate a simple linear dataset y=xb+e
##### store validation data from same dataset too

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across the nout axis (the number of possibile values y is averaged over, whose probabilities sum to 1)
  - 'own_n' means same as self, but divided by count of non-masked items in second to last, nout lenght dimension
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [ ]:
import numpy as np
import kernelcompare as kc
#set training data size
trainsize=20
datagen_dict_override={}
datagen_dict_override['train_n']=trainsize
datagen_dict_override['ftype']='quadratic'

opt_dict_override={}
modeldict={}
hyper_param_form_dict={}
hyper_param_dict={}
opt_settings_dict={}
options={}

modeldict['Ndiff_type']='recursive'
modeldict['max_bw_Ndiff']=2
modeldict['ykern_grid']=trainsize+1
#modeldict['hyper_param_form_dict']={'y_bandscale':'fixed'}
#hyper_param_dict['y_bandscale']=np.array([1])
#opt_dict_override['hyper_param_dict']=hyper_param_dict
opt_dict_override['modeldict']=modeldict

options['fatol']=0.5
options['xatol']=.05
opt_settings_dict['options']=options
#opt_settings_dict['help_start']='no'
#opt_settings_dict['partial_match']='no'
opt_dict_override['opt_settings_dict']=opt_settings_dict


quad_misspec=kc.DoKernelOpt(datagen_dict_override=datagen_dict_override, opt_dict_override=opt_dict_override)

print(quad_misspec.minimize_obj)
'''
optimizedict1=quad_misspec.optimizedict
x=quad_misspec.train_x
y=quad_misspec.train_y
xfull=np.concatenate([np.ones([trainsize,1]),x],axis=1)
n,p=x.shape

x_validate=quad_misspec.val_x
y_validate=quad_misspec.val_y'''

:val(20) replaces val(40)

:val(quadratic) replaces val(linear)

val is dict in modeldict, recursive call
:val(recursive) replaces val(product)

:val(2) replaces val(2)

:val(21) replaces val(50)

made it back from recursive call
val is dict in opt_settings_dict, recursive call
val is dict in options, recursive call
:val(0.5) replaces val(0.1)

:val(0.05) replaces val(0.001)

made it back from recursive call
made it back from recursive call
------no start value overrides encountered------
newoptimizedict1{'opt_settings_dict': {'method': 'Nelder-Mead', 'options': {'xatol': 0.05, 'fatol': 0.5, 'adaptive': True}, 'help_start': 1, 'partial_match': 1}, 'hyper_param_dict': {'Ndiff_exponent': array([0.3, 0.3]), 'x_bandscale': array([-0.03, -0.03]), 'outer_x_bw': array([1]), 'outer_y_bw': array([3]), 'Ndiff_depth_bw': array([0.5]), 'y_bandscale': array([2.])}, 'modeldict': {'Ndiff_type': 'recursive', 'Ndiff_start': 1, 'max_bw_Ndiff': 2, 'normalize_Ndiffwtsum': 'own_n', 'xkern_grid': 'no', 'yke

/home/douglas/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6667: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


free_params added to dict. free_params:[ 1.87336972  0.10005638 -0.16628302  0.24443755 -0.62583819  6.81700537
  5.53432076  0.580868  ]
free_params added to dict. free_params:[ 1.89066549  0.10184299 -0.15909385  0.24880222 -0.63701315  6.84656266
  5.30837331  0.58502956]
free_params added to dict. free_params:[ 1.8978191   0.09843168 -0.16885257  0.24046841 -0.61567593  7.01934642
  5.35622982  0.59252887]
saved to model_save at about 20191111-144639 with mse=268.60929944586826
free_params added to dict. free_params:[ 1.89199023  0.10121126 -0.16090102  0.24725892 -0.63306181  6.87855965
  5.31723563  0.58641832]
free_params added to dict. free_params:[ 1.89690834  0.09886599 -0.16761014  0.24152942 -0.61839247  6.99734849
  5.35013697  0.59157409]
free_params added to dict. free_params:[ 1.89010662  0.095997   -0.16848976  0.2494533  -0.63868012  6.83306392
  5.30463452  0.58444368]
free_params added to dict. free_params:[ 1.88897696  0.09516038 -0.16923435  0.25076932 -0.64204956

/home/douglas/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:1019: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)


free_params added to dict. free_params:[ 1.15111878  0.09569063 -0.24896667  0.36160221 -0.63891437  5.82575259
  3.07137304  0.55436955]
saved to model_save at about 20191111-144830 with mse=155.78684906456118
free_params added to dict. free_params:[ 1.13986532  0.09493101 -0.25206881  0.35987871 -0.65253455  5.69029827
  2.93870219  0.56242776]
free_params added to dict. free_params:[ 1.10638875  0.09597975 -0.25425988  0.36032991 -0.66571255  5.78837111
  2.88782741  0.55400597]


/home/douglas/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:1162: RuntimeWarning: overflow encountered in true_divide
  result = self.f(da, db, *args, **kwargs)


free_params added to dict. free_params:[ 1.09599576  0.09606224 -0.25494908  0.36000867 -0.67146091  5.79553172
  2.86137256  0.55276356]
free_params added to dict. free_params:[ 1.0961185   0.09735153 -0.25626084  0.36177429 -0.65002197  5.58271781
  2.910285    0.54485508]
free_params added to dict. free_params:[ 1.09930576  0.09699461 -0.25550212  0.36096665 -0.66325106  5.74367719
  2.7944065   0.54599049]
free_params added to dict. free_params:[ 1.09173059  0.09721237 -0.25611889  0.36084575 -0.66696729  5.74731504
  2.7559742   0.54382851]
free_params added to dict. free_params:[ 1.13895833  0.09814562 -0.25002512  0.35064559 -0.66323364  5.91627733
  2.95029958  0.54498191]


#-----------------Calibrate/Optimize--------

In [ ]:
import myreg
x=quad_misspec.train_x
y=quad_misspec.train_y
xfull=np.concatenate([np.ones([x.shape[0],1]),x],axis=1)
just_linreg=myreg.reg(y,xfull)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

In [ ]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

In [ ]:
#xgrid=np.linspace(1,n,n)
x=quad_misspec.train_x
y=quad_misspec.train_y


xgrid=x[:,0]
yhat=quad_misspec.optimize_obj.yhat_un_std
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=10,color='blue',alpha=0.4,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',alpha=0.9,legend='yhat')
p.circle(xgrid,just_linreg.yhat,color='red',legend='just lin reg yhat')
p.line(xgrid,np.mean(y))
p.legend.location = "top_left"
#p.yaxis.visible=False
show(p)

In [ ]:
print(yhat)
print(y)

In [ ]:

ypredict=optimized_Ndiff_kernel.predict_tool(xvalidate,optimized_Ndiff_kernel.fixed_or_free_paramdict,modeldict1)
prediction_err=yvalidate-ypredict
prediction_mse=np.mean(prediction_err**2)

In [ ]:
np.ma.count_masked(yhat)

## class toolcompare(object):